In [ ]:
import pandas as pd
import plotly.express as px
import seaborn as sns

In [ ]:
df = pd.read_csv("C:/Users/danie/Downloads/SmA-Four-Tank-Batch-Process_V2.csv", sep = ";")

In [ ]:
col = []

for x in df.columns:
    col.append(x.split(" ")[0])
    
df.columns = col

In [ ]:
# Entferne alle Schritte mit StepID = 2, da diese für das Projekt nicht relevant sind
df.drop(df[df["CuStepNo"] == 2].index, inplace = True)

In [ ]:
# Extrahiere die Zeitangaben bzw. formatiere die Zeitstempel in das richtige Format
df.timestamp = pd.to_datetime(df.timestamp)

# df['timestamp']=pd.to_datetime(df['timestamp'],format='%Y-%m-%dT%H:%M:%S.%f')

df["day"] = df.timestamp.dt.day
df["hour"] = df.timestamp.dt.hour
df["minute"] = df.timestamp.dt.minute
df["second"] = df.timestamp.dt.second

In [ ]:
df[df.any(axis = 1).isna()]

In [ ]:
# determine start and end of steps
df['dstep_p']=df['CuStepNo'].diff()
df['dstep_n']=df['CuStepNo'].diff(-1)

vsteps = [1,7,8,3]

# select rows with a step change
dfsen=df[(df['dstep_n']!=0)]
dfsep=df[(df['dstep_p']!=0)]
dfse=pd.concat([dfsen,dfsep])
dfse=dfse.sort_values(by=['timestamp'])

# create new dataframe where we store extracted information
dfinfo_steps=pd.DataFrame(columns=['step_length','start','end','stepn'])

# iterative approach
pstep=-1
c=0
for n in range(dfse.shape[0]):
    # get row
    r=dfse.iloc[n]
    if pstep==r['CuStepNo']:
        # determine step length
        stepl=r['timestamp']-dfse.iloc[n-1]['timestamp']
        # update dataframe
        dfinfo_steps.loc[c]=(stepl,dfse.iloc[n-1]['timestamp'],r['timestamp'],r['CuStepNo'])
        c=c+1
    else:
        pstep=r['CuStepNo']
print(dfinfo_steps.head())
print('Max step_length: {}'.format(dfinfo_steps['step_length'].max()))
print('Min step_length: {}'.format(dfinfo_steps['step_length'].min()))
print('#steps: {}'.format(dfinfo_steps.shape[0]))

# now determine whether the batch is complete
batchn=1
batchi=-1
dfinfo_steps["batchn"]=0
dfinfo_steps["is_complete"]=False
dfinfo_batches=pd.DataFrame(columns=['batch_length','start','end','steps','batchn','is_complete'])
n=0
b=0
while True:
    if n+len(vsteps)>dfinfo_steps.shape[0]:
        # complete info at incomplete, last batch
        steps=[]
        for v in range(dfinfo_steps.shape[0]-n):
            dfinfo_steps.at[n+v,'batchn']=batchi
            dfinfo_steps.at[n+v,'is_complete']=False
            steps.append(dfinfo_steps.at[n+v,'stepn'])
        dfinfo_batches.loc[b]=[dfinfo_steps.at[n+v,'end']-dfinfo_steps.at[n,'start'],dfinfo_steps.at[n,'start'], \
                               dfinfo_steps.at[n+v,'end'],steps,batchi,False]
        b=b+1
        break
    # check if all steps of a batch are present and in correct order
    isCorrect=True
    for v in range(len(vsteps)):
        isCorrect=dfinfo_steps.loc[n+v,'stepn']==vsteps[v]
        if not isCorrect:
            break
    if isCorrect:
        steps=[]
        for v in range(len(vsteps)):
            dfinfo_steps.at[n+v,'batchn']=batchn
            dfinfo_steps.at[n+v,'is_complete']=True
            steps.append(dfinfo_steps.at[n+v,'stepn'])
        dfinfo_batches.loc[b]=[dfinfo_steps.at[n+v,'end']-dfinfo_steps.at[n,'start'],dfinfo_steps.at[n,'start'], \
                               dfinfo_steps.at[n+v,'end'],steps,batchn,True]
        n=n+len(vsteps)
        batchn=batchn+1
        b=b+1
    else:
        steps=[]
        for vc in range(v):
            dfinfo_steps.at[n+vc,'batchn']=batchi
            dfinfo_steps.at[n+vc,'is_complete']=False
            steps.append(dfinfo_steps.at[n+v,'stepn'])
        dfinfo_batches.loc[b]=[dfinfo_steps.at[n+vc,'end']-dfinfo_steps.at[n,'start'],dfinfo_steps.at[n,'start'], \
                               dfinfo_steps.at[n+vc,'end'],steps,batchi,False]
        batchi=batchi-1
        n=n+vc
        b=b+1
# save dfinfo_steps to file
dfinfo_steps.to_pickle('SmA-Four-Tank-Info-Steps.pkl')
dfinfo_batches.to_pickle('SmA-Four-Tank-Info-Batches.pkl')


In [ ]:
df_steps = pd.read_pickle("C:/Users/danie/SmA-Four-Tank-Info-Steps.pkl")
df_batches = pd.read_pickle("C:/Users/danie/SmA-Four-Tank-Info-Batches.pkl")

In [ ]:
# Entferne den letzten unvollständigen Batch aus dem Datensatz

df = df.drop(df[df.timestamp >= '2018-10-31 14:29:32'].index)

In [ ]:
df[df.DeviationID == 0]

In [ ]:
# Entferne einen Zeileneintrag, bei dem die DeviationID 0 ist

df = df.drop(df[df.DeviationID == 0].index)

# DevId
### 1 --> Pl1200 (zwischen PL12002 und PL12003)
### 2 --> PL1100 (zwischen YC10001 und YC14001)
### 3 --> YC21006
### 4 --> YC22006
### 5 --> YC23006
### 6 --> YS14005
### 7 --> Pl2150
### 8 --> YS14004
### 9 --> YS10004

Schritt 7 generell am kürzesten, Schritt 8 am längsten (evtl. wegen Pausen zwischen den Batches? An Wochenenden wurden jedoch keine Daten gespeichert)

In [ ]:
df.groupby("CuStepNo").agg({"second":"sum"})

In [ ]:
df_steps.groupby("stepn").agg({"step_length":"sum"})

#### Wochenenden am:
13.-14.

20.-21.

27.-28.

### 16. und 23. Oktober komplett fehlerfrei

In [ ]:
df[df.DeviationID > 1].day.unique()

In [ ]:
df_steps.sort_values("step_length", ascending = False)

Batchprozesse finden zwischen 6 und 16 Uhr statt

In [ ]:
df.hour.describe()

In [ ]:
# Füge Batchnummern und Schrittlängen zum Dataframe hinzu
a = df.merge(df_steps, how = "left", left_on = "timestamp", right_on = "start")

In [ ]:
a[["stepn"]] = a[["stepn"]].fillna(method = "ffill")
a[["batchn"]] = a[["batchn"]].fillna(method = "ffill")
a[["step_length"]] = a[["step_length"]].fillna(method = "ffill")

In [ ]:
# redundante oder unnötige Spalten entfernen
a.drop(columns = ["start", "end", "is_complete", "stepn"], inplace = True)

## Ein Batchprozess benötigt hier zwischen 468 und 1997 Sekunden.

In [ ]:
a.groupby(["batchn"]).agg({"timestamp":"count", "DeviationID":"unique", "CuStepNo":"unique"}).sort_values("timestamp")

### Schritt 8 in Batch 9 äußerst kurz

In [ ]:
a[a.batchn == 9].drop_duplicates(subset = ["step_length"])

In [ ]:
a.groupby("DeviationID").agg({"batchn":"nunique"})

In [ ]:
a.groupby("day").agg({"batchn":"nunique"})

### Bei 9 Spalten sind die gemessenene Werte konstant

--> nicht nützlich zur Anomaliedetektion/ Klassifizierung, kann entfernt werden

In [ ]:
useless_columns = []

for x in a.columns:
    if a[x].describe()[2] == 0.0:
        print(x)
        useless_columns.append(x)
        print(f"unique values: {a[x].unique()}")
        
a.drop(columns = useless_columns, inplace = True)

In [ ]:
a["DeviationID"] = a["DeviationID"].astype(str)

px.scatter(a.drop_duplicates(subset = ["step_length", "batchn", "CuStepNo"]).sort_values(["batchn","DeviationID"]), 
           x = "step_length", 
           y = "batchn", 
           facet_row = "CuStepNo", title = "Dauer pro Schritt in jedem Batch",
          color = "DeviationID")

In [ ]:
a["step_length_total_seconds"] = a.step_length.dt.total_seconds()

px.scatter(a.drop_duplicates(["batchn", "CuStepNo"]), 
           x = "step_length_total_seconds", y = "DeviationID", 
           facet_col = "CuStepNo",
          title = "Dauern bestimmte Schritte länger bei Fehlern?",
          hover_name = "batchn")

In [ ]:
# Immer ein bestimmter Wert für DeviationID pro Batch
a.groupby("batchn").agg({"DeviationID":"nunique"}).sort_values("DeviationID")

In [ ]:
df_30s = a.resample("30S", on = "timestamp").mean()

In [ ]:
df_30s = df_30s.reset_index()

In [ ]:
df_30s.iloc[:, 1:32].columns

In [ ]:
a["DeviationID"] = a.DeviationID.astype(float)

# Datenpunkte in Minutenintervalle zusammenfassen
a.resample("60S", on = "timestamp").mean().reset_index()

In [ ]:
px.histogram(a.resample("5S", on = "timestamp").mean().reset_index(), x = "batchn", nbins = 300,
            title = "Wurden alle Batches gleich oft im Datensatz gespeichert?", color = "DeviationID", width = 1000)

In [ ]:
px.histogram(a.resample("10S", on = "timestamp").mean().reset_index(), x = "timestamp", nbins = 300, color = "DeviationID")

## Überblick über alle gemessenen Werte

manche Fehlertypen evtl. nur durch Analysieren der zeitlichen Abstände erkennbar

##### LevelMainTank um die Hälfte geringer ab 24.10.?

##### PI12002_PV_Out ungefähr am 16.10. auf 0 ?

##### FIC14002_SP kurz 0 am 26./27.10. --> Neustart des Systems?

### DevID = 2.0 oder 3.0 
--> YC14001_MV, FIC14002_MV nur auf 100?

### DevID = 4.0 
--> FIC14002_PV_Out über 5000, 

FIC14002_MV unter 30, 

YC14006_MV sehr hoch, 

PI14012_PV_Out sehr hoch (auch bei DevID = 3.0)

PIC14007_MV sehr hoch


### DevID = 8.0 
--> YC14006_MV deutlich niedriger, 

PI14012_PV_Out unter 0 (beide Bauteile hängen miteinander zusammen), 

YC23001_MV auf 100, 

FIC23002_MV auf 100, 

FIC14002_SP minimal

In [ ]:
for x in a.iloc[:, 3:33].columns:
    plt.figure(figsize=(15, 5), dpi=80)
    plt.plot(a["timestamp"], a[x])
    plt.title(x)

In [ ]:
a[a.FIC23002_MV == 100].DeviationID.unique()

In [ ]:
a[a.YC23001_MV == 100].DeviationID.unique()

In [ ]:
a[a.FIC14002_SP == a.FIC14002_SP.min()].DeviationID.unique()

In [ ]:
a[a.PI14012_PV_Out < 0].DeviationID.unique()

In [ ]:
a[a.FIC14002_PV_Out > 5000].DeviationID.unique()

In [ ]:
a[a.PIC14007_MV >= 40].DeviationID.unique()

In [ ]:
a[a.FIC14002_MV < 30].DeviationID.unique()

In [ ]:
a[a.YC14006_MV == a.YC14006_MV.max()]

In [ ]:
a[a.PI14012_PV_Out > 2.0].DeviationID.unique()

## zu DeviationID = 2.0

In [ ]:
px.line(a[["timestamp", "DeviationID", "YC14001_MV", "day"]][a.day.isin([16,17,18])],
       x = "timestamp", y = "YC14001_MV", facet_row = "DeviationID")

In [ ]:
px.line(a[["timestamp", "DeviationID", "FIC14002_MV"]][a.day.isin([16,17,18])],
       x = "timestamp", y = "FIC14002_MV", 
        facet_row = "DeviationID")